# Strategy design pattern

The strategy design pattern allows the selection of an algorithm using a common interface. First, we will see how to implement it using classes and afterwards, we will simplify it by using first order functions.

## Strategy using classes

In [ ]:
from abc import ABC, abstractclassmethod
from collections.abc import Sequence
from decimal import Decimal
from typing import NamedTuple, Optional

class Customer(NamedTuple):
    name: str
    fidelity: int

class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self) -> Decimal: # Question: we do not type annotate self?
        return self.price * self.quantity

class Order(NamedTuple): # The context
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional['Promotion'] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0)) # Why the start param?

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self) -> str:
        return f'Order total: {self.total():.2f} due: {self.due():.2f}'

class Promotion(ABC): # the Strategy: an abstract base class
    @abstractclassmethod
    def discount(self, order: Order) -> Decimal:
        """Return discount as a positive dollar amount"""

class FidelityPromo(Promotion):
    """5% discount for customers with 1000 or more fidelity points"""
    

    def discount(self, order: Order) -> Decimal:
        rate = Decimal('0.05')
        if order.customer.fidelity >= 1000:
            return order.total() * rate
        return Decimal(0)
        

class BulkItemPromo(Promotion):
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order: Order) -> Decimal:
        discount = Decimal(0)
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * Decimal('0.1')
        return discount

class LargeOrderPromo(Promotion):
    """7% discount for orders with 10 or more distinct items"""


    def discount(self, order: Order) -> Decimal:
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * Decimal('0.07')
        return Decimal(0)


In [ ]:
# Test of class defined above

joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = (LineItem('banana', 4, Decimal('0.5')),
        LineItem('apple', 10, Decimal('1.5')),
        LineItem('watermelon', 5, Decimal(5)))
    
Order(joe, cart, FidelityPromo())
Order(ann, cart, FidelityPromo())

banana_cart = (LineItem('banana', 30, Decimal('.5')),
               LineItem('banana', 30, Decimal('.5')))

Order(joe, banana_cart, BulkItemPromo())
long_cart = tuple(LineItem(str(sku), 1, Decimal(1)) for sku in range(10))
Order(joe, long_cart, LargeOrderPromo())

## Strategy using first order functions

In [ ]:
from collections.abc import Sequence
from dataclasses import dataclass
from decimal import Decimal
from typing import Optional, Callable, NamedTuple


class Customer(NamedTuple):
    name: str
    fidelity: int


class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self):
        return self.price * self.quantity
    
@dataclass(frozen=True)
class Order: # The context
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional[Callable[['Order'], Decimal]] = None


    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self) -> str:
        return f'<Order total: {self.total():.2f} due: {self.due():.2f}'

def fidelity_promo(order: Order) -> Decimal:
    """5% discount for customers with 1000 or more fidelity points"""
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal('0.05')
    return Decimal(0)

def bulk_item_promo(order: Order) -> Decimal:
    """10% discount for each LineItem with 20 or more units"""
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal('0.1')
    return discount


def large_order_promo(order: Order) -> Decimal:
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal('0.07')
    return Decimal(0)

In [ ]:
# Test of the strategy using first order functions

joe = Customer('John Doe', 0)
ann = Customer('John Doe', 1100)

cart = [LineItem('banana', 4, Decimal('0.5')),
        LineItem('apple', 10, Decimal('1.5')),
        LineItem('watermelon', 5, Decimal(5))]
    
Order(ann, cart, fidelity_promo) # <Order total: 42.00 due: 42.00>
Order(ann, cart, fidelity_promo) # <Order total: 42.00 due: 39.90>

banana_cart = (LineItem('banana', 30, Decimal('.5')),
               LineItem('banana', 30, Decimal('.5')))

Order(joe, banana_cart, bulk_item_promo) #  <Order total: 30.00 due: 28.50>
long_cart = tuple(LineItem(str(sku), 1, Decimal(1)) for sku in range(10))
Order(joe, long_cart, large_order_promo) # <Order total: 10.00 due: 9.30>

# Choosing the best promotion
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in promos)

Order(joe, long_cart, best_promo)

Notice that every time a new promo is implemented, the list promos needs to be updated. We can solve this issue using the `globals()` function, that returns all the objects available in the global namespace.

In [ ]:
promos = [promo for name, promo in globals().items()
                if name.endswith('_promo') and name != 'best_promo'
]

Another option would be defining a module (file) `promotions.py` and leave there just the implementations of the promotions. Using the function `inpect.getmembers(promotions, inspect.isfunction)` we could get all functions of that module.

A even more robust solution would be to implement the Strategy Design Pattern using decorators:

In [ ]:
Promotion = Callable[[Order], Decimal]
promos: list[Promotion] = []

def promotion(promo: Promotion) -> Promotion:
    promos.append(promo)
    return promo

def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in promos) 

@promotion
def fidelity(order: Order) -> Decimal:
    pass

Notice that using this strategy, we do not need to have a `_promo` suffix in the function to be able to identify it. Furthermore, every time a function uses the `@promotion` decorator, the function will be added to the `promos` registry. You can even remove a promo from the list, by just removing the decorator and running again your code.

# The Command Design Pattern

The command pattern is close to the Strategy pattern, but in this case we will have a `Invoker` that will call a `Command` and this will result in an action on a `Receiver`. 

Question: The implementation of the commands are functions here also?

Below, we can have a `MacroCommand` that is responsible of executing a list of commands

In [ ]:
from typing import Any


class MacroCommand:
    """A command that executes a list of commands"""

    def __init__(self, commands) -> None:
        self.commands = list(commands)
    
    def __call__(self) -> Any:
        for command in self.commands:
            command()